In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import numpy as np
from Models import MoELSTM
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

from typing import List, Tuple, Optional, Dict
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
import random
from Models import MoELSTM, LSTMModel, train_model
from Preprocess import (
    compute_metrics,
    convert_timeseries_to_numpy,
    create_dataloader,
    load_building_series,
    split_series_list,
)
import pandas as pd
from collections import defaultdict
import os
import torch
import torch.optim as optim
from tqdm import tqdm


from Models import model_fn
from tqdm import tqdm
from my_utils import train_model, load_energy_data_feather, get_weights, set_weights


In [2]:
from AggregationStrategy import sync_aggregate,average_weights,sync_aggregate_norm,sync_aggregate_softmax, fedavgm_update

In [4]:
df = pd.read_feather("train_final.feather")

In [5]:
df.head()

,building_id,meter,timestamp,meter_reading,primary_use,air_temperature
7593144,0,0,2016-05-21 01:00:00,72.221012,Education,25.6
7593145,1,0,2016-05-21 01:00:00,39.611586,Education,25.6
7593146,2,0,2016-05-21 01:00:00,1.920567,Education,25.6
7593147,3,0,2016-05-21 01:00:00,111.532464,Education,25.6
7593148,4,0,2016-05-21 01:00:00,456.734799,Education,25.6


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11712248 entries, 7593144 to 20216099
Data columns (total 6 columns):
 #   Column           Dtype         
---  ------           -----         
 0   building_id      int64         
 1   meter            int64         
 2   timestamp        datetime64[ns]
 3   meter_reading    float64       
 4   primary_use      object        
 5   air_temperature  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(1)
memory usage: 625.5+ MB


In [7]:


# Config
# List of models to experiment with
MODEL_NAMES = ["cnn-lstm", "cnn-gru"]

# Config
NUM_CLIENTS = 1410
CLIENT_FRAC = 0.15
NUM_ROUNDS = 50
LOCAL_EPOCHS = 5
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATA_FILE ="train_final.feather" # "meter_0_data_cleaned.feather"


### FedProx

In [8]:
def train_model_fedprox(
    model,
    train_loader,
    global_weights,
    device=None,
    learning_rate=0.001,
    loss_fn=None,
    optimizer_class=optim.Adam,
    epochs=50,
    mu=0.01
):
    """Train model using FedProx loss (with proximal term)."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    loss_fn = loss_fn or nn.MSELoss()
    optimizer = optimizer_class(model.parameters(), lr=learning_rate)
    loss_history = []

    # Clone global weights as tensors on the correct device
    # prox_ref = [torch.tensor(w, dtype=torch.float32, device=device) for w in global_weights]
    prox_ref = [w.detach().clone().to(device) for w in global_weights]

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            if y_batch.dim() == 3 and y_batch.shape[-1] == 1:
                y_batch = y_batch.squeeze(-1)

            optimizer.zero_grad()
            output = model(X_batch)
            task_loss = loss_fn(output, y_batch)

            # Compute proximal term
            prox_term = 0.0
            for p, w_ref in zip(model.parameters(), prox_ref):
                prox_term += torch.norm(p - w_ref) ** 2
            prox_term = (mu / 2) * prox_term

            total_loss = task_loss + prox_term
            total_loss.backward()
            optimizer.step()

            epoch_loss += total_loss.item()

        loss_history.append(epoch_loss / len(train_loader))

    return get_weights(model), loss_history

In [9]:
for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")

    # Directory to save checkpoints
    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    # Init model and weights
    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")
        sampled_clients = random.sample(range(NUM_CLIENTS), k=int(CLIENT_FRAC * NUM_CLIENTS))
        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, loss_history = train_model_fedprox(
                model=local_model,
                train_loader=train_loader,
                global_weights=[torch.tensor(w).to(DEVICE) for w in global_weights],
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS,
                mu=0.01  # optional, default used if not passed
            )
            local_weights.append(updated_weights)

        # FedProx uses FedAvg-style aggregation
        global_weights = average_weights(local_weights)
        set_weights(global_model, global_weights)

        # Save model checkpoint
        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_fedProx.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: cnn-lstm
Round 1/50


Training clients: 100%|██████████| 211/211 [02:35<00:00,  1.36it/s]


Saved global model to results/cnn-lstm/cnn-lstm_round_1_fedProx.pt
Round 2/50


Training clients: 100%|██████████| 211/211 [02:38<00:00,  1.33it/s]


Saved global model to results/cnn-lstm/cnn-lstm_round_2_fedProx.pt
Round 3/50


...

Training clients: 100%|██████████| 211/211 [02:41<00:00,  1.31it/s]


Saved global model to results/cnn-lstm/cnn-lstm_round_49_fedProx.pt
Round 50/50


Training clients: 100%|██████████| 211/211 [02:37<00:00,  1.34it/s]

Saved global model to results/cnn-lstm/cnn-lstm_round_50_fedProx.pt
Starting experiment with model: cnn-gru


ValueError: Unknown model name: cnn-gru

In [ ]:
class TimeSeriesDifficultyWeight:
    def __init__(self, num_clients, accumulate_iters=20):
        self.num_clients = num_clients
        self.last_loss = torch.ones(num_clients).float().to(DEVICE)
        self.learn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.unlearn_score = torch.zeros(num_clients).float().to(DEVICE)
        self.ema_difficulty = torch.ones(num_clients).float().to(DEVICE)
        self.accumulate_iters = accumulate_iters

    def update(self, cid: int, loss_history: List[float]) -> float:
        """
        Update difficulty based on loss trend for a client.
        Expects a list of per-epoch losses.
        """
        current_loss = torch.tensor(loss_history[-1], dtype=torch.float32).to(DEVICE)
        previous_loss = self.last_loss[cid]
        delta = current_loss - previous_loss
        ratio = torch.log((current_loss + 1e-8) / (previous_loss + 1e-8))

        learn = torch.where(delta < 0, -delta * ratio, torch.tensor(0.0, device=current_loss.device))
        unlearn = torch.where(delta >= 0, delta * ratio, torch.tensor(0.0, device=current_loss.device))

        # EMA update
        momentum = (self.accumulate_iters - 1) / self.accumulate_iters
        self.learn_score[cid] = momentum * self.learn_score[cid] + (1 - momentum) * learn
        self.unlearn_score[cid] = momentum * self.unlearn_score[cid] + (1 - momentum) * unlearn

        # Difficulty score
        diff_ratio = (self.unlearn_score[cid] + 1e-8) / (self.learn_score[cid] + 1e-8)
        difficulty = diff_ratio #torch.pow(diff_ratio, 1 / 5)

        # Smooth difficulty over rounds
        self.ema_difficulty[cid] = momentum * self.ema_difficulty[cid] + (1 - momentum) * difficulty

        self.last_loss[cid] = current_loss
        return self.ema_difficulty[cid].item()

    def get_normalized_weights(self, client_ids: List[int]) -> List[float]:
        weights = [self.ema_difficulty[cid].item() for cid in client_ids]
        total = sum(weights)
        if total == 0:
            return [1.0 / len(client_ids)] * len(client_ids)
        return [w / total for w in weights]
    
    def get_sampling_probabilities(self, min_prob=0.05):
        difficulty = self.ema_difficulty
        inv_difficulty = 1.0 / (difficulty + 1e-6)
        inv_difficulty = inv_difficulty / inv_difficulty.sum()
        probs = torch.clamp(inv_difficulty, min=min_prob)
        return (probs / probs.sum()).cpu().numpy()



In [ ]:
# from my_utils import TimeSeriesDifficultyWeight  # make sure it's imported



for model_name in MODEL_NAMES:
    print(f"Starting experiment with model: {model_name}")
    difficulty_tracker = TimeSeriesDifficultyWeight(num_clients=NUM_CLIENTS)

    model_dir = os.path.join("results", model_name)
    os.makedirs(model_dir, exist_ok=True)

    global_model = model_fn(model_name).to(DEVICE)
    global_weights = get_weights(global_model)

    for rnd in range(NUM_ROUNDS):
        print(f"Round {rnd+1}/{NUM_ROUNDS}")

        # === Difficulty-aware sampling ===
        sampling_probs = difficulty_tracker.get_sampling_probabilities(min_prob=0.05)
        sampled_clients = np.random.choice(
            np.arange(NUM_CLIENTS),
            size=int(CLIENT_FRAC * NUM_CLIENTS),
            replace=False,
            p=sampling_probs
        )
        print(f"Sampled {len(sampled_clients)} clients")

        local_weights = []

        for cid in tqdm(sampled_clients, desc="Training clients"):
            local_model = model_fn(model_name).to(DEVICE)
            set_weights(local_model, global_weights)
            train_loader, test_loader = load_energy_data_feather(cid, filepath=DATA_FILE)

            updated_weights, loss_history = train_model_fedprox(
                model=local_model,
                train_loader=train_loader,
                global_weights=[torch.tensor(w).to(DEVICE) for w in global_weights],
                device=DEVICE,
                learning_rate=LR,
                loss_fn=None,
                optimizer_class=optim.Adam,
                epochs=LOCAL_EPOCHS,
                mu=0.01
            )

            # === Update difficulty score ===
            difficulty_tracker.update(cid, loss_history)

            local_weights.append(updated_weights)

        # === FedAvg-style aggregation ===
        global_weights = average_weights(local_weights)
        set_weights(global_model, global_weights)

        checkpoint_path = os.path.join(model_dir, f"{model_name}_round_{rnd+1}_fedProx_diff.pt")
        torch.save(global_model.state_dict(), checkpoint_path)
        print(f"Saved global model to {checkpoint_path}")


Starting experiment with model: lstm
Round 1/50
Sampled 211 clients


Training clients:   0%|          | 0/211 [00:00<?, ?it/s]

Training clients: 100%|██████████| 211/211 [03:36<00:00,  1.03s/it]


Saved global model to results/lstm/lstm_round_1_fedProx_diff.pt
Round 2/50
Sampled 211 clients


Training clients: 100%|██████████| 211/211 [03:34<00:00,  1.02s/it]


...

Training clients: 100%|██████████| 211/211 [03:30<00:00,  1.00it/s]


Saved global model to results/gru/gru_round_49_fedProx_diff.pt
Round 50/50
Sampled 211 clients


Training clients: 100%|██████████| 211/211 [03:33<00:00,  1.01s/it]

Saved global model to results/gru/gru_round_50_fedProx_diff.pt


: 